In [1]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
cancer_data = load_breast_cancer()
X = cancer_data.data
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)

(455, 30)
(114, 30)


In [3]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=1000)

lr_final = LogisticRegression(C=10)

In [4]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

In [5]:
print(accuracy_score(y_test, knn_pred))
print(accuracy_score(y_test, rf_pred))
print(accuracy_score(y_test, dt_pred))
print(accuracy_score(y_test, ada_pred))

0.9210526315789473
0.9649122807017544
0.9122807017543859
1.0


In [6]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


In [7]:
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


In [9]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print(accuracy_score(y_test, final))

1.0


CV 세트 기반 스태킹

In [16]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False)
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, ' model 시작')

    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t 폴드 세트:', folder_counter, ' 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        model.fit(X_tr, y_tr)

        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)

    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    return train_fold_pred, test_pred_mean

In [17]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작
	 폴드 세트: 0  시작
	 폴드 세트: 1  시작
	 폴드 세트: 2  시작
	 폴드 세트: 3  시작
	 폴드 세트: 4  시작
	 폴드 세트: 5  시작
	 폴드 세트: 6  시작
RandomForestClassifier  model 시작
	 폴드 세트: 0  시작
	 폴드 세트: 1  시작
	 폴드 세트: 2  시작
	 폴드 세트: 3  시작
	 폴드 세트: 4  시작
	 폴드 세트: 5  시작
	 폴드 세트: 6  시작
DecisionTreeClassifier  model 시작
	 폴드 세트: 0  시작
	 폴드 세트: 1  시작
	 폴드 세트: 2  시작
	 폴드 세트: 3  시작
	 폴드 세트: 4  시작
	 폴드 세트: 5  시작
	 폴드 세트: 6  시작
AdaBoostClassifier  model 시작
	 폴드 세트: 0  시작
	 폴드 세트: 1  시작
	 폴드 세트: 2  시작
	 폴드 세트: 3  시작
	 폴드 세트: 4  시작
	 폴드 세트: 5  시작
	 폴드 세트: 6  시작


In [18]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print(X_train.shape)
print(X_test.shape)
print(Stack_final_X_train.shape)
print(Stack_final_X_test.shape)

(455, 30)
(114, 30)
(455, 4)
(114, 4)


In [20]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print(accuracy_score(y_test, stack_final))

0.9824561403508771
